<a href="https://colab.research.google.com/github/RyosukeHanaoka/TechTeacher/blob/main/hand_image_segmenter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hand Image Segmenter

## Imports and Setup
Let's start with the base imports for running this MediaPipe sample.

In [ ]:
!pip install -q mediapipe==0.10.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Download the image segmenter model

The next thing you will need to do is download the image segmentation model that will be used for this demo. In this case you will use the DeepLab v3 model.

In [ ]:
#@title Start downloading here.
!wget -O deeplabv3.tflite -q https://storage.googleapis.com/mediapipe-models/image_segmenter/deeplab_v3/float32/1/deeplab_v3.tflite

## Download a test image

After downloading the model, it's time to grab an image that you can use for testing! It's worth noting that while this is working with a single image, you can download a collection of images to store in the `IMAGE_FILENAMES` array.

In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# 入力および出力ディレクトリを設定
base_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/'
output_base_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/segmented_photo/'
sub_dirs = ['RA/RA_Right_Hand', 'RA/RA_Left_Hand', 'nonRA/nonRA_Right_Hand', 'nonRA/nonRA_Left_Hand']
output_sub_dirs = ['RA/RA_Right_Hand', 'RA/RA_Left_Hand', 'nonRA/nonRA_Right_Hand', 'nonRA/nonRA_Left_Hand']

# 画像のパスを取得
image_paths = []
for sub_dir in sub_dirs:
    full_dir = os.path.join(base_dir, sub_dir)
    for filename in os.listdir(full_dir):
        if filename.endswith('.jpg'):
            image_path = os.path.join(full_dir, filename)
            image_paths.append(image_path)

# 出力ディレクトリを作成
for sub_dir in output_sub_dirs:
    os.makedirs(os.path.join(output_base_dir, sub_dir), exist_ok=True)

WHITE_COLOR = (255, 255, 255)  # white

# ImageSegmenterのオプションを作成
base_options = python.BaseOptions(model_asset_path='deeplabv3.tflite')
options = vision.ImageSegmenterOptions(base_options=base_options, output_category_mask=False)#次回はoutput_category_mask=Falseで試みる。

# ImageSegmenterを作成
with vision.ImageSegmenter.create_from_options(options) as segmenter:
    for image_path in image_paths:
        image = mp.Image.create_from_file(image_path)
        segmentation_result = segmenter.segment(image)
        category_mask = segmentation_result.category_mask

        image_data = image.numpy_view()

        # Apply the mask to set the background to white
        white_bg_image = np.zeros(image_data.shape, dtype=np.uint8)
        white_bg_image[:] = WHITE_COLOR
        condition = np.stack((category_mask.numpy_view(),) * 3, axis=-1) > 0.2
        output_image = np.where(condition, image_data, white_bg_image)

        print(f'Segmentation mask of {os.path.basename(image_path)}:')

        output_sub_dir = [d for d in output_sub_dirs if d in image_path][0]
        output_path = os.path.join(output_base_dir, output_sub_dir, os.path.basename(image_path))
        cv2.imwrite(output_path, output_image)


Segmentation mask of patients_000006.jpg:
Segmentation mask of patients_000008.jpg:
Segmentation mask of patients_000009.jpg:
Segmentation mask of patients_000010.jpg:
Segmentation mask of patients_000011.jpg:
Segmentation mask of patients_000012.jpg:
Segmentation mask of patients_000013.jpg:
Segmentation mask of patients_000014.jpg:
Segmentation mask of patients_000015.jpg:
Segmentation mask of patients_000016.jpg:
Segmentation mask of patients_000017.jpg:
Segmentation mask of patients_000018.jpg:
Segmentation mask of patients_000019.jpg:
Segmentation mask of patients_000020.jpg:
Segmentation mask of patients_000021.jpg:
Segmentation mask of patients_000022.jpg:
Segmentation mask of patients_000023.jpg:
Segmentation mask of patients_000024.jpg:
Segmentation mask of patients_000025.jpg:
Segmentation mask of patients_000026.jpg:
Segmentation mask of patients_000027.jpg:
Segmentation mask of patients_000028.jpg:
Segmentation mask of patients_000029.jpg:
Segmentation mask of patients_0000